In [1]:
import os
import ee
import geemap
from pathlib import Path
from datetime import datetime, timedelta , date
import time
import pandas as pd
import csv
import glob
import folium
from IPython.display import display, clear_output
import shutil
import shapefile
import ipywidgets as widgets
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [5]:
shp = shapefile.Reader("".join(my_button.files))
b = []
for i in range(len(shp.fields)):
    a = shp.fields[i][0]
    b.append(a)

# give the shapefile name

file_name = widgets.Dropdown(
options=b,
description='Regional category')

file_name

Dropdown(description='Regional category', options=('DeletionFlag', 'Shape_Leng', 'Shape_Area', 'ID'), value='D…

In [6]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

In [7]:
# give the bands
#
band_name = widgets.SelectMultiple(
options=['Air_2m_T_C_mean','Air_2m_T_C_min','Air_2m_T_C_max','dewpoint_2m_C','RH','mean_2m_air_temperature','minimum_2m_air_temperature',
        'maximum_2m_air_temperature','dewpoint_2m_temperature','total_precipitation','surface_pressure','mean_sea_level_pressure',
        'u_component_of_wind_10m','v_component_of_wind_10m'],
description='Band',
)

band_name

SelectMultiple(description='Band', options=('Air_2m_T_C_mean', 'Air_2m_T_C_min', 'Air_2m_T_C_max', 'dewpoint_2…

In [8]:
statics =widgets.Dropdown(
    options=['MEAN','MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'],
    value='MEAN',
    description='Statistics')

statics

Dropdown(description='Statistics', options=('MEAN', 'MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'),…

In [9]:
# give the output floder and flie name
folder = FileChooser()
display(folder)

FileChooser(path='C:\Users\Yang Hsiu\Downloads\satellite python code-20220518T062225Z-001\satellite python cod…

In [10]:
# create folder
folder_name = 'data_all'

# create folder name : data_all
if os.path.isdir(folder_name) == True:
    shutil.rmtree(folder_name, ignore_errors=True)
    os.makedirs(folder_name)
else:
    os.makedirs(folder_name)


In [11]:
states = geemap.shp_to_ee("".join(my_button.files))

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def last_day_of_year(any_day):
    next_year = any_day.replace(year=any_day.year, month=12, day = 31)  # this will never fail
    return next_year

def yearlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.year < end.year:
            next_year = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_year = end
        if next_year == end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_year(begin).strftime("%Y-%m-%d")])
        begin = next_year
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def trans_date(input_date):
    t1 = input_date
    t2 = datetime.strptime(t1, '%Y%m%d').strftime('%Y/%m/%d')
    return t2

In [12]:
def getC_air_mean(image):
    
    Air_2m_T_C_mean = image.expression(
      'T - 273.15',{
      'T' : image.select('mean_2m_air_temperature')}).rename('Air_2m_T_C_mean');
    
    return image.addBands(Air_2m_T_C_mean)

def getC_air_min(image):
    
    Air_2m_T_C_min = image.expression(
      'T - 273.15',{
      'T' : image.select('minimum_2m_air_temperature')}).rename('Air_2m_T_C_min');
    
    return image.addBands(Air_2m_T_C_min)

def getC_air_max(image):
    
    Air_2m_T_C_max = image.expression(
      'T - 273.15',{
      'T' : image.select('maximum_2m_air_temperature')}).rename('Air_2m_T_C_max');
    
    return image.addBands(Air_2m_T_C_max)


def getC_dewpoint(image):
    
    dewpoint_2m_temperature_C = image.expression(
      'T - 273.15',{
      'T' : image.select('dewpoint_2m_temperature')}).rename('dewpoint_2m_C');
    
    return image.addBands(dewpoint_2m_temperature_C)

def getRH(image):
    
    RH = image.expression(
      '100 * (exp((17.625 * Td)/(243.04 + Td))/exp((17.625*T)/(243.04 + T)))',{
      'Td' : image.select('dewpoint_2m_temperature').add(-273.15),
      'T' : image.select('mean_2m_air_temperature').add(-273.15)}).rename('RH');
    
    return image.addBands(RH)

def cbind(statics):

    all_files = glob.glob(os.path.join(folder_name,"era5_{}*.csv".format(statics)))

    df_from_each_file = (pd.read_csv(f, sep = ",") for f in all_files)
    df_merged = pd.concat(df_from_each_file, ignore_index = True)
    if 'Air_2m_T_C_mean' in df_merged.columns.tolist():
        df_merged.rename(columns={'Air_2m_T_C_mean' : 'Air_2m_T_C_mean_' + str(statics)}, inplace = True)
    else:
        pass
    if 'Air_2m_T_C_min' in df_merged.columns.tolist():
        df_merged.rename(columns={'Air_2m_T_C_min' : 'Air_2m_T_C_min_' + str(statics)}, inplace = True)
    else:
        pass
    if 'Air_2m_T_C_max' in df_merged.columns.tolist():
        df_merged.rename(columns={'Air_2m_T_C_max' : 'Air_2m_T_C_max_' + str(statics)}, inplace = True)
    else:
        pass
    if 'dewpoint_2m_C' in df_merged.columns.tolist():
        df_merged.rename(columns={'dewpoint_2m_C' : 'dewpoint_2m_C_' + str(statics)}, inplace = True)
    else:
        pass
    if 'RH' in df_merged.columns.tolist():
        df_merged.rename(columns={'RH' : 'RH_' + str(statics)}, inplace = True)
    else:
        pass
    if 'mean_2m_air_temperature' in df_merged.columns.tolist():
        df_merged.rename(columns={'mean_2m_air_temperature' : 'mean_2m_air_temperature_' + str(statics)}, inplace = True)
    else:
        pass
    if 'minimum_2m_air_temperature' in df_merged.columns.tolist():
        df_merged.rename(columns={'minimum_2m_air_temperature' : 'minimum_2m_air_temperature_' + str(statics)}, inplace = True)
    else:
        pass
    if 'maximum_2m_air_temperature' in df_merged.columns.tolist():
        df_merged.rename(columns={'maximum_2m_air_temperature' : 'maximum_2m_air_temperature_' + str(statics)}, inplace = True)
    else:
        pass
    if 'dewpoint_2m_temperature' in df_merged.columns.tolist():
        df_merged.rename(columns={'dewpoint_2m_temperature' : 'dewpoint_2m_temperature_' + str(statics)}, inplace = True)
    else:
        pass
    if 'total_precipitation' in df_merged.columns.tolist():
        df_merged.rename(columns={'total_precipitation' : 'total_precipitation_' + str(statics)}, inplace = True)
    else:
        pass
    if 'surface_pressure' in df_merged.columns.tolist():
        df_merged.rename(columns={'surface_pressure' : 'surface_pressure_' + str(statics)}, inplace = True)
    else:
        pass
    if 'mean_sea_level_pressure' in df_merged.columns.tolist():
        df_merged.rename(columns={'mean_sea_level_pressure' : 'mean_sea_level_pressure_' + str(statics)}, inplace = True)
    else:
        pass
    if 'u_component_of_wind_10m' in df_merged.columns.tolist():
        df_merged.rename(columns={'u_component_of_wind_10m' : 'u_component_of_wind_10m_' + str(statics)}, inplace = True)
    else:
        pass
    if 'v_component_of_wind_10m' in df_merged.columns.tolist():
        df_merged.rename(columns={'v_component_of_wind_10m' : 'v_component_of_wind_10m_' + str(statics)}, inplace = True)
    else:
        pass

    df_merged.to_csv(folder.selected + '.csv')
    
    shutil.rmtree(folder_name, ignore_errors=True)

""" 
Note that here, the merged file path cannot be the same as the single file path, and must be placed in different folders, 
so this time the following line of code is to output the files to another folder.
"""


' \nNote that here, the merged file path cannot be the same as the single file path, and must be placed in different folders, \nso this time the following line of code is to output the files to another folder.\n'

In [13]:
def zonal(statics):
    time_list =monthlist(star.value, end.value)
    for i in range(0,len(time_list)):
        star_time = time.time()
        
        if os.path.isfile(folder_name +'era5_{}_{}.csv'.format(statics,time_list[i])) == True:
                
                continue
            
        else:
            clear_output(wait=True)
            
            era5 = ee.ImageCollection("ECMWF/ERA5/DAILY") \
                  .filter(ee.Filter.date(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1))) \
                  .filterBounds(states) \
                  .map(getRH) \
                  .map(getC_air_mean) \
                  .map(getC_air_min) \
                  .map(getC_air_max) \
                  .map(getC_dewpoint) \
                  .select(list(band_name.value)) \
                  .map(lambda image: image.clip(states))


            era5 = era5.toBands()
            
            out_dir = os.path.expanduser(folder_name)
            out_dem_stats = os.path.join(out_dir, 'era5_{}_{}.csv'.format(statics,time_list[i]))

            if not os.path.exists(out_dir):
                os.makedirs(out_dir)

            geemap.zonal_statistics(era5, states, out_dem_stats, statistics_type=statics, scale=1000)
            
            data_temp = pd.read_csv(out_dem_stats)
            column_name_list = data_temp.columns.tolist()
            c = []
            d = []
            for k in zip(column_name_list[:]):
                c.append(k[0][0:8])
                d.append(k[0])
                
            data = []
            for j in range(0, len(column_name_list),len(band_name.value)):            
                if all(m.isdigit() for m in c[j:j+len(band_name.value)]) == True:
                        
                    df = data_temp.loc[:,d[j:j+len(band_name.value)]]
                    df[file_name.value] = data_temp.loc[:,[file_name.value]]
                    text = column_name_list[j][0:8]
                    df.insert(0, 'Date', '')
                    df['Date'] = trans_date(text)
                    df.insert(1, 'Doy', '')
                    df['Doy'] = datetime.strptime(text, '%Y%m%d').strftime('%j')
                    colnames=['Date','Doy']
                    colnames.extend(list(band_name.value))
                    colnames.append(file_name.value)
                    df.columns=[colnames]
                    data.append(df)
                else:
                    continue
            appended_data = pd.concat(data, axis=0,ignore_index = True)
            #cols = appended_data.columns.to_list()
            #cols.insert(len(appended_data.columns), cols.pop(cols.index(file_name)))
            #appended_data = appended_data[cols]
                
            appended_data.to_csv(out_dem_stats,index=False)#Output the file with date and doy back
            
            end_time = time.time()
            print(end_time - star_time)
            time.sleep(1)

In [ ]:
zonal(statics.value)
cbind(statics.value)

Computing statistics ...
Generating URL ...
